# Statistics by country
Get the top 10 countries, based on the data copied from the "Visitors domains/countries" page http://emodnet01.cineca.it/oceanbrowser/cgi-bin/awstats/awstats.pl?month=11&year=2018&output=main&config=sdn-oceanbrowser-net&framename=index (change month and year for other periods).

In [1]:
from bs4 import BeautifulSoup
import os
import glob
import logging
import re
import pycountry

ModuleNotFoundError: No module named 'pycountry'

In [77]:
logger = logging.getLogger("awstats")
logger.setLevel(logging.INFO)
logging.info("starting")

In [4]:
datadir = "../Statistics/"
filelist = sorted(glob.glob(os.path.join(datadir, "countries*.dat")))
nfiles = len(filelist)
logger.info("Working on {} files".format(nfiles))

INFO:awstats:Working on 12 files


In [16]:
class CountryStat(object):
    
    def __init__(self, name=None, pages=0, hits=0, bw=0):
        self.name = name
        self.pages = pages
        self.hits = hits
        self.bw = bw
        
    def __repr__(self):
        return "Country {0}: {1} pages, {2} hits and {3} MB of bandwidth".format(self.name, 
                                                                          self.pages,
                                                                          self.hits,
                                                                          self.bw)

In [58]:
def findcountry(cname, clist):
    """
    Find the object in a list which has the attribute 'name' matching cname
    """
    index = None
    for i, c in enumerate(clist):
        if c.name == cname:
            index = i
            break
    return index

In [78]:
clist = []
countrylist = []
for statfiles in filelist:
    logger.info("Working on file {}".format(os.path.basename(statfiles)))
    
    with open(statfiles) as f:
        line = ""

        # Read until BEGIN_DOMAIN is found
        while not "Domains/Countries" in line:
            line = f.readline().rstrip()
            
        # Continue the reading until the end
        ii = 0
        
        
        for line in f:
            
            if "Others" in line:
                break
            
            line = line.rstrip()

            if (ii % 3) == 0:
                countryname = line.split()[1]
                pages = int(line.split()[-1])
            if (ii % 3) == 1:
                hits = int(line.split()[-1])
            if (ii % 3) == 2:
                bandwidth = float(line.split()[1])
            
                ind = findcountry(countryname, clist)
                if ind is None:
                    logger.debug("New country")
                    countrylist.append(countryname)
                    c = CountryStat(countryname, pages, hits, bandwidth)
                    clist.append(c)
                else:
                    clist[ind].pages += pages
                    clist[ind].hits += hits
                    clist[ind].bw += bandwidth                  
            ii += 1

INFO:awstats:Working on file countries201811.dat
INFO:awstats:Working on file countries201812.dat
INFO:awstats:Working on file countries201901.dat
INFO:awstats:Working on file countries201902.dat
INFO:awstats:Working on file countries201903.dat
INFO:awstats:Working on file countries201904.dat
INFO:awstats:Working on file countries201905.dat
INFO:awstats:Working on file countries201906.dat
INFO:awstats:Working on file countries201907.dat
INFO:awstats:Working on file countries201908.dat
INFO:awstats:Working on file countries201909.dat
INFO:awstats:Working on file countries201910.dat


In [82]:
clist_sorted = sorted(clist, key=lambda x: x.pages, reverse=True)

In [86]:
fo = open("../Statistics/ocean_browser_statsCountry2019.csv", 'w')
fo.write("\t".join(("country name", "Pages", "Hits", "\n")))
for i in range(0, 12):
    country = clist_sorted[i]
    fo.write("\t".join((country.name, str(country.pages), str(country.hits), "\n")))
fo.close()